In [1]:
import geemap
import ee
ee.Initialize(project='pyregence-ee')
import scripts.analysis_functions as af
import scripts.fire_info as fi
import scripts.get_image_collections as gic
import scripts.utils as utils

In [2]:
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/nifc/nifc_fires_2021_20211215")
acre_min = 500
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

fires_w_filters = gic.remove_recent(fires).filter(ee.Filter.gte('Acres', acre_min))
print(f'Removing very recent fires... removing fires less than {acre_min} acres')
print(f'Computing Burn Severity for {fires_w_filters.size().getInfo()} fires')

run_mode = fi.config_mode(fires)
print(f'run mode: {run_mode}')

def bs_calc(feat: ee.Feature):
    fire = ee.Feature(feat)
    # pass one fire feature into new fire_info function
    region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature_v2(fire, run_mode)

    
    # key consideration for the realtime mode is how to handle more recent fires that may have cloudy imagery... 
    # can put date filter in gic.remove_recent() further back from today's date so all fires left in the FeatureColl should have enough images to 
    # get a good .mean() composite, or keep the date filter pretty close to today's date and let it pull T1 realtime toa imgs with no cloudBusting (as is done currently) 
    # and just caveat that the more recent fire perims will have less reliable BS metric, and its easy to run this every few months to get a better severity estimate on those fires
    sensor = "landsat"
    pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
    pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

    if run_mode == 'historical':
        post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
    else: # realtime 
        post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bestls') # these now passed as None

    post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end) # using make_pre_composite to use the imgColl.mean() composite instead of firstNonNull, since a lot of fires in a year have enough img for a good analysis
    
    rdnbr = af.rdnbr(pre_img,post_img)
    miller = af.miller_thresholds(rdnbr)
    
    return ee.Image(miller).clip(region).select('MillersThresholds').toByte()

bs_imgColl = ee.FeatureCollection(fires_w_filters).map(bs_calc)
bs_img = ee.ImageCollection(bs_imgColl).max().rename('SEVERITY')

Total Fires in FeatureCollection: 4892
Removing very recent fires... removing fires less than 500 acres
Computing Burn Severity for 578 fires
run mode: realtime


In [3]:
Map = geemap.Map()

Map.addLayer(bs_img, {"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs img')
#Map.addLayer(featCollv1, {'color':'pink'}, 'fires', True,0.4)

# Map.addLayer(pre_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"pre")
# Map.addLayer(post_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"post")

# Map.addLayer(region, {}, 'region')
# Map.centerObject(region, 10)


# Map.addLayer(miller.clip(region),{"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
Map.setCenter(-118.6633, 36.2576, 7)
Map

Map(center=[36.2576, -118.6633], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

## Looking under the hood at one fire 

In [17]:
# Look under the hood for a given fire
run_mode = fi.config_mode(fires)
print(f'run mode: {run_mode}')

#def bs_calc(feat: ee.Feature):
fire = ee.FeatureCollection(fires_w_filters.filter(ee.Filter.eq('Name', 'Dixie'))).sort('Acres',False).first()
print(fire.getInfo()['properties'])
# pass one fire feature into new fire_info function
region,pre_start,pre_end,fire_start,fire_end = fi.get_fire_info_from_feature_v2(fire, run_mode)
print(pre_start.getInfo())
print(pre_end.getInfo())
print(fire_start.getInfo())
print(fire_end.getInfo())

# key consideration for the realtime mode is how to handle more recent fires that may have cloudy imagery... 
# can put date filter in gic.remove_recent() further back from today's date so all fires left in the FeatureColl should have enough images to 
# get a good .mean() composite, or keep the date filter pretty close to today's date and let it pull T1 realtime toa imgs with no cloudBusting (as is done currently) 
# and just caveat that the more recent fire perims will have less reliable BS metric, and its easy to run this every few months to get a better severity estimate on those fires
sensor = "landsat"
pre_collection = gic.get_image_collection(sensor,region,pre_start,pre_end,landsatCol='bestls',cloudBustingMethod='bust')
pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)
print(pre_collection.size().getInfo())
print(pre_img.bandNames().getInfo())

if run_mode == 'historical':
    post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bust')
else: # realtime 
    post_collection = gic.get_image_collection(sensor,region,fire_start,fire_end,landsatCol='bestls', cloudBustingMethod='bestls') # these now passed as None

post_img = gic.get_composite(post_collection,gic.make_pre_composite,fire_start, fire_end) # using make_pre_composite to use the imgColl.mean() composite instead of firstNonNull, since a lot of fires in a year have enough img for a good analysis
print(post_collection.size().getInfo())
print(post_img.bandNames().getInfo())

rdnbr = af.rdnbr(pre_img,post_img)
miller = af.miller_thresholds(rdnbr)

#return ee.Image(miller).clip(region).select('MillersThresholds').toByte()

# bs_imgColl = ee.FeatureCollection(fires_w_filters).map(bs_calc)
# bs_img = ee.ImageCollection(bs_imgColl).max().rename('SEVERITY')

run mode: realtime
2021-07-13
2020-7-13
2020-10-11
2021-07-13
2021-10-11
15
['ca', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'temp1', 'temp2', 'BQA', 'date']
15
['ca', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cirrus', 'temp1', 'temp2', 'BQA', 'date']


In [18]:
Map = geemap.Map()
Map.addLayer(pre_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"pre")
Map.addLayer(post_img,{"bands":["swir2","nir","red"],"min":0,"max":0.848},"post")
Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
Map.addLayer(miller.clip(region),{"min":0,"max":3,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
Map.addLayer(region, {}, 'region')
Map.centerObject(region, 8)
Map



Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# export img to asset
# desc = 'bs_2020fires_gte500ac_v1'
# utils.exportImgtoAsset(bs_finalv1, desc=desc, export=True)

# desc = 'bs_2020fires_gte500ac_v2'
# utils.exportImgtoAsset(bs_finalv2, desc=desc, export=True)

# desc = 'bs_2020fires_gte500ac_v3'
# utils.exportImgtoAsset(bs_finalv3, desc=desc, export=True)

desc = 'bs_2020fires_gte500ac_2021_11_16'
utils.exportImgtoAsset(bs_nifc_plus_calfire, desc=desc, export=True)